In [ ]:
import pymongo
import json
import pandas as pd
import os 
from datetime import datetime, timedelta
import utm

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)

try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

### RETO 1

Se ha realizado una exploración visual de los datos. Se observa que la relación principal entre los archivos se encuentra en los identificadores de local. Los locales, por su lado, tienen las siguientes características en cuanto los datos proporcionados:
- Un local puede tener una terraza o no.
- Un local puede tener varias licencias, en diferentes estados.
- Un local puede tener varias actividades económicas.

Se observa lo siguiente para cada archivo:

#### Locales
Los datos estan completamente definidos en su mayoría, es decir, que el esquema es consistente, con la excepción de la hora de apertura. Dichos campos aparecen entre un 5 y 3 % de los documentos. El id_local es único para todos los documentos proporcionados.

#### Terrazas
Del mismo modo que locales, el esquema es consistente en todo caso, salvo los campos que indican los horarios de apertura en periodo de fin de semana. El id es único para toda la colección.

#### Actividad economica
Se trata de las actividades que realizan los locales. Esquema consistente, aunque hay algunos valores de coordenada (10%) que son defectuosos, valor 0.

#### Licencias
Se tratade las licencias de los locales. Esquema consistente, aunque hay algunos valores de coordenada (3%) que son defectuosos, valor 0.

Se organizará la base de datos creando una capa RAW,siguiendo una capa TRANSFORMED y una final DATAMART, que será labase de datos donde se ubicarán las colecciones finales de consulta.

In [ ]:

file_path = "./data"

# Load into raw database
raw_db = client["raw"]

# LOAD
for (_, _, filenames) in os.walk("./data"):
    for name in filenames:
        if name.endswith(".json"):

            collection_name = name.replace(".json","")

            if collection_name in raw_db.list_collection_names():
                continue

            with open(os.path.join(file_path, name)) as file:
                raw = json.load(file)

            Collection = raw_db[collection_name]

            Collection.insert_many(raw)




### RETO 2

En las siguientes celdas se realizan agregaciones para construir colecciones intermedias y las colecciones finales de consulta. Se creará una colección principal de locales_complete, disponible en datamart, que tendrá embebidos los documentos con la información significativa de la terraza asociada al local (si existe), la actividad económica que este ejerce y las licencias que ostenta. 

In [ ]:
# TRANSFORMATIONS: Actividad Economica
# Creación de datamart de agrupación de actividades economicas y referencia los locales que realizan dichas actividades.
# Se incluye el dato de en que barrio y distrito son más ocurrentes dicha actividad económica.

pipeline = [
    {
        "$group": {
            "_id": {
                "seccion": "$id_seccion",
                "division": "$id_division",
                "epigrafe": "$id_epigrafe",
                "desc_seccion": "$desc_seccion",
                "desc_division": "$desc_division",
                "desc_epigrafe": "$desc_epigrafe",
                "distrito": "$id_distrito_local",
                "barrio": "$id_barrio_local"
            },
            "count": {"$sum": 1},
            "local_ids": {"$push": {"id_local": "$id_local"}}
        }
    },
    {
        "$sort": {
            "_id.seccion": 1, "_id.division": 1, "_id.epigrafe": 1, "count": -1
        }
    },
    {
        "$group": {
            "_id": {
                "seccion": "$_id.seccion",
                "division": "$_id.division",
                "epigrafe": "$_id.epigrafe",
                "desc_seccion": "$_id.desc_seccion",
                "desc_division": "$_id.desc_division",
                "desc_epigrafe": "$_id.desc_epigrafe"
            },
            "distritos": {"$push": {"distrito": "$_id.distrito", "count": "$count"}},
            "barrios": {"$push": {"barrio": "$_id.barrio", "count": "$count"}},
            "all_local_ids": {"$push": "$local_ids"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "seccion": "$_id.seccion",
            "division": "$_id.division",
            "epigrafe": "$_id.epigrafe",
            "desc_seccion": "$_id.desc_seccion",
            "desc_division": "$_id.desc_division",
            "desc_epigrafe": "$_id.desc_epigrafe",
            "mostFrequentDistrito": {"$arrayElemAt": ["$distritos", 0]},
            "mostFrequentBarrio": {"$arrayElemAt": ["$barrios", 0]},
            "local_ids": "$all_local_ids"
        }
    },
    {
        "$merge": {
            "into": {"db": "datamart", "coll": "actividades_economicas"},
            "whenMatched": "replace",
            "whenNotMatched": "insert",
        }
    }
]

raw_db.actividadeconomica.aggregate(pipeline)


In [ ]:
# TRANSFORMATIONS: Actividad economica
# Agrupación por local de la actividad

pipeline = [
    {
        "$group": {
            "_id": "$id_local",
            "id_local": {"$first": "$id_local"},
            "actividades": {
                "$push": {
                    "id_seccion": "$id_seccion",
                    "desc_seccion": "$desc_seccion",
                    "id_division": "$id_division",
                    "desc_division": "$desc_division",
                    "id_epigrafe": "$id_epigrafe",
                    "desc_epigrafe": "$desc_epigrafe",
                }
            }
        }
    },
    {
        "$merge": {
            "into": {"db": "transformed", "coll": "actividadeconomica_by_local"},
            "on": "_id",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

raw_db.actividadeconomica.aggregate(pipeline)

In [ ]:
# TRANSFORMATIONS: Licencias
# Creación de datamart con la información de tipologia de licencia y referencia a los locales cuyo estado de licencia se encuentra en trámite. 

pipeline = [
    {
        "$group": {
            "_id": "$id_tipo_licencia",
            "total": {"$sum": 1},
            "desc_tipo_licencia": {"$first": "$desc_tipo_licencia"},
            "local_ids_en_tramite": {
                "$push": {
                    "$cond": [
                        {"$eq": ["$id_tipo_situacion_licencia", "01"]},
                        {"id_local": "$id_local"},
                        "$$REMOVE" 
                    ]
                }
            }
        }
    },
    {
        "$sort": {"total": -1}
    },
    {
        "$merge": {
            "into": {"db": "datamart", "coll": "licencia_totals"},
            "on": "_id",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

raw_db.licencias.aggregate(pipeline)


In [ ]:
# TRANSFORMATIONS: Licencias
# Transformación preiacon los agrupados de licencia por local 

raw_db = client["raw"]

pipeline = [
    {
        "$group": {
            "_id": "$id_local",
            "id_local": {"$first": "$id_local"},
            "licencias": {
                "$push": {
                    "ref_licencia": "$ref_licencia",
                    "id_tipo_licencia": "$id_tipo_licencia",
                    "desc_tipo_licencia": "$desc_tipo_licencia",
                    "id_tipo_situacion_licencia": "$id_tipo_situacion_licencia",
                    "desc_tipo_situacion_licencia": "$desc_tipo_situacion_licencia",
                    "Fecha_Dec_Lic": "$Fecha_Dec_Lic",
                    "rotulo": "$rotulo",
                    "fx_carga": "$fx_carga",
                    "fx_datos_ini": "$fx_datos_ini",
                    "fx_datos_fin": "$fx_datos_fin"
                }
            }
        }
    },
    {
        "$merge": {
            "into": {"db": "transformed", "coll": "licencias_by_local"},
            "on": "_id",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

raw_db.licencias.aggregate(pipeline)


In [ ]:
# TRANSFORMATIONS: Locales
# Transformación de unión 

transformed_db = client['transformed']

raw_db.locales.create_index([('id_local', 1)])
raw_db.terrazas.create_index([('id_local', 1)])

pipeline = [
    {
        "$lookup": {
            "from": "terrazas",
            "let": {"localId": "$id_local"},
            "pipeline": [
                {
                    "$match": {
                        "$expr": {
                            "$eq": ["$id_local", "$$localId"]
                        }
                    }
                },
                {
                    "$project": {
                        "_id": 0,
                        "id_periodo_terraza": 1,
                        "desc_periodo_terraza": 1,
                        "id_situacion_terraza": 1,
                        "desc_situacion_terraza": 1,
                        "Superficie_ES": 1,
                        "Superficie_RA": 1,
                        "Fecha_confir_ult_decreto_resol": 1,
                        "id_ndp_terraza": 1,
                        "id_clase_ndp_terraza": 1,
                        "id_vial": 1,
                        "desc_clase": 1,
                        "desc_nombre": 1,
                        "nom_terraza": 1,
                        "num_terraza": 1,
                        "cal_terraza": 1,
                        "desc_ubicacion_terraza": 1,
                        "hora_ini_LJ_es": 1,
                        "hora_fin_LJ_es": 1,
                        "hora_ini_LJ_ra": 1,
                        "hora_fin_LJ_ra": 1,
                        "hora_ini_VS_es": 1,
                        "hora_fin_VS_es": 1,
                        "hora_ini_VS_ra": 1,
                        "hora_fin_VS_ra": 1,
                        "mesas_aux_es": 1,
                        "mesas_aux_ra": 1,
                        "mesas_es": 1,
                        "mesas_ra": 1,
                        "sillas_es": 1,
                        "sillas_ra": 1,
                        "cal_edificio": 1,
                        "fx_carga": 1,
                        "fx_datos_ini": 1,
                        "fx_datos_fin": 1
                    }
                }
            ],
            "as": "terraza_info"
        }
    },
    {
        "$merge": {
            "into": {"db": "transformed", "coll": "locales_terraza"},
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

raw_db.locales.aggregate(pipeline)

In [ ]:
# TRANSFORMATIONS: Locales
# Creación del datamart de locales_complete, con la información embebida de terrazas, actividades y licencias agrupadas. 
# Se utilizan las capas de transformed.

pipeline = [
    {
        "$set": {
            "_id": "$id_local" 
        }
    },
    {
        "$lookup": {
            "from": "licencias_by_local",
            "localField": "id_local",
            "foreignField": "_id",
            "as": "licencias_info"
        }
    },
    {
        "$set": {
            "licencias_info": {"$arrayElemAt": ["$licencias_info.licencias", 0]}
        }
    },
    {
        "$lookup": {
            "from": "actividadeconomica_by_local",
            "localField": "id_local",
            "foreignField": "_id",
            "as": "actividades_info"
        }
    },
    {
        "$set": {
            "actividades_info": {"$arrayElemAt": ["$actividades_info.actividades", 0]}
        }
    },
    {
        "$merge": {
            "into": {"db": "datamart", "coll": "locales_complete"},
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

transformed_db.locales_terraza.aggregate(pipeline)


In [ ]:
# Datamart
# Creación de datamart de resumen por distrito y barrio. Se tienen referencias a los locales,
# dado que el _id de locales complete es el id_local.

pipeline = [
    {
        "$group": {
            "_id": {
                "distrito": "$id_distrito_local",
                "barrio": "$id_barrio_local",
                "seccion": "$actividades_info.id_seccion",
                "division": "$actividades_info.id_division",
                "epigrafe": "$actividades_info.id_epigrafe"
            },
            "totalLocales": {"$sum": 1},
            "totalTerrazas": {"$sum": {"$cond": [{"$gt": [{"$size": "$terraza_info"}, 0]}, 1, 0]}},
            "countActividad": {"$sum": 1}
        }
    },
    {
        "$sort": {"_id.distrito": 1, "_id.barrio": 1, "countActividad": -1}
    },
    {
        "$group": {
            "_id": {
                "distrito": "$_id.distrito",
                "barrio": "$_id.barrio"
            },
            "actividades": {"$push": {"actividad": "$_id", "count": "$countActividad"}},
            "totalLocales": {"$first": "$totalLocales"},
            "totalTerrazas": {"$first": "$totalTerrazas"}
        }
    },
    {
        "$addFields": {
            "mostFrequentActividadBarrio": {"$arrayElemAt": ["$actividades", 0]}
        }
    },
    {
        "$group": {
            "_id": "$_id.distrito",
            "barrios": {
                "$push": {
                    "barrio": "$_id.barrio",
                    "totalLocales": "$totalLocales",
                    "totalTerrazas": "$totalTerrazas",
                    "mostFrequentActividad": "$mostFrequentActividadBarrio"
                }
            },
            "actividadesDistrito": {"$push": "$mostFrequentActividadBarrio.actividad"},
        }
    },
    {
        "$project": {
            "_id": 0,
            "distrito": "$_id",
            "barrios": 1,
        }
    },
    {
        "$merge": {
            "into": {"db": "datamart", "coll": "summary_barrio_distrito"},
            "whenMatched": "replace",
            "whenNotMatched": "insert",
        }
    }
]


client['datamart'].locales_complete.aggregate(pipeline)


In [ ]:
# CREAR LAS CONSULTAS
# a.

projection = {
    "_id": 0,
    "distrito": 1,
    "barrios.barrio": 1,
    "barrios.totalLocales": 1,
    "barrios.totalTerrazas": 1
}

documents = list(client['datamart']['summary_barrio_distrito'].find({}, projection))

for doc in documents:
    print(doc)

In [ ]:
# b.
results = client['datamart']['licencia_totals'].find({}, {'desc_tipo_licencia': 1, 'total': 1, '_id': 0})
for result in results:
    print(result)

In [56]:
# c.

pipeline = [
    {"$unwind": "$local_ids_en_tramite"},
    {"$group": {"_id": None, "local_ids_en_tramite": {"$addToSet": "$local_ids_en_tramite"}}},
]

results = client['datamart']['licencia_totals'].aggregate(pipeline)

for result in results:
    print(result['local_ids_en_tramite'])

[{'id_local': 270217841}, {'id_local': 270554865}, {'id_local': 285014552}, {'id_local': 280072206}, {'id_local': 90001235}, {'id_local': 270084343}, {'id_local': 270152199}, {'id_local': 270270702}, {'id_local': 60000724}, {'id_local': 30001342}, {'id_local': 280011293}, {'id_local': 270281703}, {'id_local': 280054199}, {'id_local': 280019946}, {'id_local': 280000425}, {'id_local': 280037253}, {'id_local': 280070022}, {'id_local': 285031058}, {'id_local': 200000727}, {'id_local': 270261105}, {'id_local': 280003500}, {'id_local': 150001980}, {'id_local': 285034323}, {'id_local': 270191395}, {'id_local': 40000547}, {'id_local': 280060895}, {'id_local': 80000780}, {'id_local': 285033083}, {'id_local': 270263570}, {'id_local': 285005108}, {'id_local': 270313333}, {'id_local': 285046621}, {'id_local': 270169433}, {'id_local': 280050833}, {'id_local': 270244967}, {'id_local': 270570753}, {'id_local': 280059368}, {'id_local': 285021494}, {'id_local': 280016808}, {'id_local': 285004123}, {'id

In [57]:
# e.

projection = {
    "_id": 0,
    "distrito": 1,
    "barrios.barrio": 1,
    "barrios.mostFrequentActividad": 1
}

documents = list(client['datamart']['summary_barrio_distrito'].find({}, projection))

for doc in documents:
    print(f"Distrito: {doc['distrito']}")
    for barrio in doc['barrios']:
        print(f"  Barrio: {barrio.get('barrio', 'N/A')}, Most Frequent Actividad: {barrio.get('mostFrequentActividad', 'N/A')}")

KeyError: 'distrito'

In [ ]:
# f.

# Locales con terraza del barrio 203 tiene que abrir a las 7 y cierran a las 3.
# Se modifica el datamart de datos que contienen la informacion de los locales. 
# Dicha modificacion no afecta al resto de datamarts.

filtro = {
    "id_barrio_local": 203,
    "terraza_info": {"$exists": True, "$ne": []}  
}

update = {
    "$set": {
        "hora_apertura1": "03:00:00",
        "hora_cierre2": "07:00:00"
    }
}

result = client['datamart']['locales_complete'].update_many(filtro, update)

locales_with_terraza = client['datamart']['locales_complete'].find(filtro)

for local in locales_with_terraza:
    print(local['hora_apertura1'])
    print(local['hora_cierre2'])

Amplicación del modelo incluyendo alojamientos.

Extracción y transformación de los datos en crudo de alojamiento. Se cargan en la base de datos. Se introduce indice en location de tipo 2dsphere.

In [ ]:
## RETO 2
def load_csv_to_dataframe(filepath):
    try:
        df = pd.read_csv(filepath)
        return df
    except Exception as e:
        print(f"An error occurred while loading the CSV file: {e}")
        return None

csv_path = './data/listings.csv'
dataframe = load_csv_to_dataframe(csv_path)
dataframe['location'] = dataframe.apply(lambda row: {'type': 'Point', 'coordinates': [row['longitude'], row['latitude']]}, axis=1)
dataframe['barrio'] = dataframe["neighbourhood_cleansed"].str.upper()
dataframe['distrito'] = dataframe["neighbourhood_group_cleansed"].str.upper()

alojamientos_raw = raw_db["alojamientos"]


alojamientos_raw.delete_many({})
alojamientos_raw.insert_many(dataframe.to_dict('records'))
alojamientos_raw.create_index([("location", "2dsphere")])


Se incluye el objeto location en actividad economica.

In [ ]:
# Transform x and y coordinates to latitude/longitude of actividades economicas

def convert_to_longitude_latitude(coord_x, coord_y, zone_number, zone_letter):
    lat, lon = utm.to_latlon(coord_x, coord_y, zone_number, zone_letter)
    return lon, lat

documents = raw_db['actividadeconomica'].find({})

for document in documents:
    if document["coordenada_x_local"] == 0 or document["coordenada_y_local"] == 0:
        continue
    lon, lat = convert_to_longitude_latitude(
        document["coordenada_x_local"],
        document['coordenada_y_local'],
        30,
        'T')
    location = {
        "type": "Point",
        "coordinates": [lon, lat]
    }

    raw_db['actividadeconomica'].update_one(
        {"_id": document['_id']}, 
        {"$set": {"location": location}}
    )

alojamientos_raw = raw_db["alojamientos"]

raw_db['actividadeconomica'].create_index([("location", "2dsphere")])

Se añaden los locales cercanos con actividad comercial a los datos en crudo de los alojamientos. Se podría haber utilizado la colección agrupada de actividad economica por local, pero no tenía presente la información de localización en su esquema. Se tiene que local cercano es aquel que se encuentra a 100 metros.

In [ ]:
# Fetch all alojamientos
alojamientos = raw_db.alojamientos.find()

for alojamiento in alojamientos:
    aloja_location = alojamiento['location']

    nearby_locales = raw_db.actividadeconomica.aggregate([
        {
            "$geoNear": {
                "near": aloja_location,
                "distanceField": "distance",
                "maxDistance": 100,
                "spherical": True
            }
        },
        {
            "$match": {
                "id_seccion": "G"
            }
        },
        {
            "$group": {
                "_id": "$id_local",
                "count": {"$sum": 1} 
            }
        },
        {
            "$project": {"id_local": "$_id", "_id": 0}  
        }
    ])

    nearby_locales_ids = [locale['id_local'] for locale in nearby_locales]
    raw_db.alojamientos.update_one(
        {"_id": alojamiento['_id']},
        {"$set": {"near_locales": nearby_locales_ids}}
    )

Agrupamiento por distrito y barrio de los alojamientos en capa TRANSFORMED. Se realiza de nuevo el summary_barrio_distrito considerando las nuevas consultas a satisfacer. Las siguientes transformaciones van en este sentido. 

In [ ]:
# TRANSFORMATION: Group alojamientos by barrio and distrito

pipeline = [
    {
        "$group": {
            "_id": {
                "distrito": "$distrito",
                "barrio": "$barrio"
            },
            "alojamientos": {
                "$push": {
                    "_id": "$_id",
                    "beds": "$beds",
                    "number_of_reviews": "$number_of_reviews",
                    "review_scores_rating": "$review_scores_rating",
                    "price": "$price",
                    "room_type": "$room_type",
                    "location": "$location",
                    "near_locales": "$near_locales"
                }
            },
            "total_alojamientos_per_barrio": {"$sum": 1}
        }
    },
    {
        "$group": {
            "_id": "$_id.distrito",
            "barrios": {
                "$push": {
                    "_id": "$_id.barrio",
                    "barrio": "$_id.barrio",
                    "alojamientos": "$alojamientos",
                    "total_alojamientos_per_barrio": "$total_alojamientos_per_barrio"
                }
            },
            "total_alojamientos_per_distrito": {"$sum": "$total_alojamientos_per_barrio"}
        }
    },
    {
        "$merge": {
            "into": {"db": "transformed", "coll": "alojamientos_by_distrito_barrio"},
            "on": "_id",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

raw_db.alojamientos.aggregate(pipeline)



Agrupamiento por distrito y barrio de las terrazas y locales en capa TRANSFORMED.

In [ ]:
# TRANSFORMATION: Group locales_terraza by barrio and distrito
# Use datamart for the transformed collection

two_years_ago = datetime.now() - timedelta(days=2*365)

pipeline = [
    {
         "$addFields": {
            "meetsCriteria": {
                "$gt": [
                    {
                        "$size": {
                            "$filter": {
                                "input": {"$ifNull": ["$licencias_info", []]},
                                "as": "lic",
                                "cond": {
                                    "$and": [
                                        {"$eq": ["$$lic.id_tipo_situacion_licencia", "02"]},
                                        {
                                            "$gte": [
                                                {
                                                    "$dateFromString": {
                                                        "dateString": "$$lic.Fecha_Dec_Lic",
                                                        "format": "%d/%m/%Y"
                                                    }
                                                },
                                                two_years_ago
                                            ]
                                        }
                                    ]
                                }
                            }
                        }
                    },
                    0
                ]
            }
        }
    },
    {
        "$group": {
            "_id": {
                "distrito":  { "$trim": { "input": "$desc_distrito_local" } },
                "barrio": { "$trim": { "input": "$desc_barrio_local" } }
            },
            "locales": {
                "$push": {
                    "_id": "$_id",
                    "terraza_info": "$terraza_info",
                    "actividades_info": "$actividades_info",
                    "licencias_info": "$licencias_info",
                    "coordenada_x_local": "$coordenada_x_local",
                    "coordenada_y_local": "$coordenada_y_local"
                }
            },
            "total_locales_per_barrio": {"$sum": 1},
            "total_terrazas_per_barrio": {
                "$sum": {
                    "$cond": [{ "$ne": ["$terraza_info", None] }, 1, 0]
                }
            },
             "total_terrazas_2_años_licencia": {
                "$sum": {
                    "$cond": [
                        {"$and": [{"$ne": ["$terraza_info", None]}, "$meetsCriteria"]},
                        1,
                        0
                    ]
                }
            }
        }
    },
    {
        "$group": {
            "_id": "$_id.distrito",
            "barrios": {
                "$push": {
                    "_id":"$_id.barrio",
                    "barrio": "$_id.barrio",
                    "locales": "$locales",
                    "total_locales_per_barrio": "$total_locales_per_barrio",
                    "total_terrazas_per_barrio": "$total_terrazas_per_barrio",
                    "total_terrazas_2_años_licencia": "$total_terrazas_2_años_licencia"
                }
            },
            "total_locales_per_distrito": {"$sum": "$total_locales_per_barrio"},
            "total_terrazas_per_distrito": {"$sum": "$total_terrazas_per_barrio"}
        }
    },
    {
        "$merge": {
            "into":  {"db": "transformed", "coll": "locales_terraza_by_distrito_barrio"},
            "on": "_id",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

client['datamart']['locales_complete'].aggregate(pipeline)

Creación del datamart resumen de información de distrito y barrio, que responderá las consultas a. y b. 

In [ ]:
# CREATE DATAMART USING PYTHON 

locales_data = {doc['_id']: doc for doc in client['transformed'].locales_terraza_by_distrito_barrio.find()}
alojamientos_data = {doc['_id']: doc for doc in client['transformed'].alojamientos_by_distrito_barrio.find()}

merged_data = []

all_distritos = set(locales_data.keys()) | set(alojamientos_data.keys())

for distrito in all_distritos:
    locales_doc = locales_data.get(distrito, {'total_locales_per_distrito': 0, 'barrios': []})
    alojamientos_doc = alojamientos_data.get(distrito, {'total_alojamientos_per_distrito': 0, 'barrios': []})

    merged_doc = {
        '_id': distrito,
        'total_locales_per_distrito': locales_doc.get('total_locales_per_distrito', 0),
        'total_alojamientos_per_distrito': alojamientos_doc.get('total_alojamientos_per_distrito', 0),
        'barrios': []
    }

    barrios_dict = {}

    for barrio in locales_doc.get('barrios', []):
        barrio_id = barrio['_id']
        barrio_info = {
            '_id': barrio_id,
            'barrio': barrio['barrio'],
            'total_locales_per_barrio': barrio.get('total_locales_per_barrio', 0),
            'total_terrazas_per_barrio': barrio.get('total_terrazas_per_barrio', 0), 
            'total_terrazas_2_años_licencia': barrio.get('total_terrazas_2_años_licencia', 0), 
            'total_alojamientos_per_barrio': len(barrio.get('alojamientos', []))
        }
        barrios_dict[barrio_id] = barrio_info

    for barrio in alojamientos_doc.get('barrios', []):
        barrio_id = barrio['_id']
        if barrio_id in barrios_dict:
            barrios_dict[barrio_id]['total_alojamientos_per_barrio'] = barrio.get('total_alojamientos_per_barrio', 0)
        else:
            barrios_dict[barrio_id] = {
                '_id': barrio_id,
                'barrio': barrio['barrio'],
                'total_alojamientos_per_barrio': barrio.get('total_alojamientos_per_barrio', 0),
                'total_locales_per_barrio': 0,
                'total_terrazas_per_barrio': 0, 
            }

    merged_doc['barrios'] = list(barrios_dict.values())

    merged_data.append(merged_doc)

client['datamart'].summary_barrio_distrito.delete_many({})
client['datamart'].summary_barrio_distrito.insert_many(merged_data)


Consulta a.

In [ ]:
client['datamart'].summary_barrio_distrito.find_one()

Consulta b.

In [ ]:

pipeline = [
    {"$unwind": "$barrios"},

    {"$sort": {"barrios.total_alojamientos_per_barrio": -1, "barrios.total_terrazas_2_años_licencia": -1}},

    {"$project": {
        "_id": 0,
        "barrio": "$barrios._id",
        "total_alojamientos_per_barrio": "$barrios.total_alojamientos_per_barrio",
        "total_terrazas_2_años_licencia": "$barrios.total_terrazas_2_años_licencia"
    }},
]

results = client['datamart'].summary_barrio_distrito.aggregate(pipeline)

for result in results:
    print(result)

Creación de datamart para responder consulta c.

In [ ]:
# CREATE DATAMART
import numpy as np

locales_terraza = client['transformed']['locales_terraza_by_distrito_barrio']
alojamientos = client['transformed']['alojamientos_by_distrito_barrio']

datamart_final = client['datamart']['alojamientos_reseña']
datamart_final.delete_many({})

for doc in alojamientos.find():
    for barrio in doc['barrios']:
        alojamientos_barrio = barrio['alojamientos']
        
        reviews = [aloj['number_of_reviews'] for aloj in alojamientos_barrio if aloj.get('number_of_reviews')]
        # Convertir precios a float, manejar cadenas que no son números válidos como NaN
        precios = [
            float(precio.replace("$", "").replace(",", ""))
            if isinstance(precio, str) else np.nan
            for precio in (aloj.get('price') for aloj in alojamientos_barrio)
            if precio is not None
        ]
        # Filtrar NaN después de la conversión
        precios = [precio for precio in precios if not np.isnan(precio)]
        
        reviews = [aloj['number_of_reviews'] for aloj in alojamientos_barrio if 'number_of_reviews' in aloj]
        camas = [aloj['beds'] for aloj in alojamientos_barrio if 'beds' in aloj]

        print(alojamientos_barrio)
        print(camas)

        precio_promedio = np.mean(precios) if precios else 0
        reviews_promedio = np.mean(reviews) if reviews else 0
        camas_promedio = np.mean(camas) if camas else 0
        
        if not reviews:
            continue
        percentil_80 = np.percentile(reviews, 80)
        
        alojamientos_filtrados = [aloj for aloj in alojamientos_barrio if aloj.get('number_of_reviews', 0) >= percentil_80]
        
        doc_insertar = {
            "_id": barrio['_id'],
            "alojamientos_con_mas_reseñas": [{
                "_id": aloj['_id'],
                "number_reviews": aloj['number_of_reviews'],
                "near_locales": aloj['near_locales']
            } for aloj in alojamientos_filtrados],
            "precio_promedio": precio_promedio,
            "score_promedio": reviews_promedio,
            "camas_promedio": camas
        }
        datamart_final.update_one({"_id": barrio['_id']}, {"$set": doc_insertar}, upsert=True)


Consulta c.

In [ ]:
# El datamart alojamientos_reseña está construido específicamente para servir a dicha consulta. 
client['datamart'].alojamientos_reseña.find_one()

Consulta d.

In [ ]:
# El datamart alojamientos_reseña está construido específicamente para servir a dicha consulta. 
# Cada barrio cuenta con la información de la media de los alojamientos ubicados en el para las características mencionadas.
# No se incluye baños ni dormitorios ya que es NaN en todo los datos.
client['datamart'].alojamientos_reseña.find_one()

## RETO 3

### Modelo de grafo

En función de las consultas específicadas, se diseña el siguiente modelo de grafo, representando los locales, terrazas y alojamientos de Madrid.

##### *Nodos y atributos*:

1. Distrito: Representación de los distritos.
- Nombre
- Identificador 

2. Barrio: Subdivisión de distrito
- Nombre
- Identificador
3. Local: Establecimientos de comercio que se encuentran en un barrio
- Rótulo 
- Identificador
- Horario Apertura
- Horario Cierre
- Coordenadas (derivadas de coordenada_x_local y coordenada_y_local)

4. Licencia: Licencia legal asociada al local
- Descripcion
- Fecha de decisión
- Identificador

5. Actividades economicas: Descripción de la actividad que realiza el local
- Sección
- División
- Epígrafe

4. Terraza: Espacio público del local
- Superficie
- Horario de apertura y cierre
- Capacidad
- Cantidad de mesas
- Cantidad de sillas 


5. Alojamiento: Lugar donde personas se hospedan
- Nombre, 
- identificador
- precio
- cantidad de dormitorios
- cantidad de baños,
- cantidad de reseñas,
- valoración general

Las etiquetas estan relacionadas con la tipología de los nodos: `:Distrito`, `:Barrio`, `:Local`, `:Licencia`, `:ActividadedEconomica`, `:Terraza`, `:Alojamiento`

#### *Relaciones y atributos*
En las relaciones aportaremos metadatos.

1. **PERTENECE_A**: Relación de `Barrio` a `Distrito`, indicando a qué distrito pertenece cada barrio.
- Sin atributos especiales. 
2. **UBICADO_EN**: Relación de `Local` y `Alojamiento` a `Barrio`, indicando en que barrio se encuentra cada establecimiento.
- Dirección del establecimiento.
- Coordenadas
3. **TIENE**: Relación de `Local` a `Terraza`, indicando que un local tiene una terraza asociada. 
- Fecha de Confirmación de la Última Resolución
- Periodo (indica estacionalidad)
4. **TIENE_LICENCIA**: Relación de `Local` a `Licencia`, indicando con que licencias cuenta el local.
- Fecha de decisión de la licencia
- Estado
5. **REALIZA**: Relación de `Local` a `ActividadEconomica`, indicando que actividades económicas realiza el local.
- Sin atributos especiales. 

Consulta todos los locales y terrazas del barrio "Salamanca":

```cypher
MATCH (b:Barrio {nombre: "Salamanca"})<-[:UBICADO_EN]-(l:Local)
OPTIONAL MATCH (l)-[:TIENE]->(t:Terraza)
RETURN l.rotulo AS Local, l.id_local AS ID_Local, t.id AS ID_Terraza, t.superficie AS Superficie, t.capacidad AS Capacidad
```

Consulta los alojamientos cuyo precio supere los 100€.

```cypher
MATCH (a:Alojamiento)
WHERE a.precio > 100
RETURN a.nombre AS Nombre, a.id_local AS ID_Alojamiento, a.precio AS Precio
```

Consulta los barrios donde los alojamientos no cuentan con dormitorios.
```cypher
MATCH (b:Barrio)<-[:UBICADO_EN]-(a:Alojamiento)
WHERE NOT EXISTS(a.cantidad_de_dormitorios) OR a.cantidad_de_dormitorios = 0
RETURN DISTINCT b.nombre AS Barrio
```

Consulta los barrios donde los alojamientos no cuentan con reseñas.
```cypher
MATCH (b:Barrio)<-[:UBICADO_EN]-(a:Alojamiento)
WHERE NOT EXISTS(a.cantidad_de_reseñas) OR a.cantidad_de_reseñas = 0
RETURN DISTINCT b.nombre AS Barrio
```


